In [ ]:
from scipy.sparse import dok_matrix
from switss.model import DTMC, ReachabilityForm

In [ ]:
P = [[0.3, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0],
     [0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5],
     [0.0, 0.1, 0.0, 0.7, 0.0, 0.2, 0.0],
     [0.0, 0.2, 0.0, 0.4, 0.4, 0.0, 0.0],
     [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
     [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     [0.0, 0.0, 0.1, 0.9, 0.0, 0.0, 0.0]]

labels = {    "target" : {3,4,6},
              "init" : {0} }

mc = DTMC(P, labels)
print(mc)
mc.digraph()

In [ ]:
reachability_form = ReachabilityForm(mc, "init", "target", new_target_label="rf_target", new_fail_label="rf_fail")
mc = reachability_form.system
mc.digraph()

In [ ]:
tra_path, lab_path = mc.save("datasets/toy")
print("saved at %s, %s" % (tra_path, lab_path))
mc_loaded = DTMC.from_file(lab_path, tra_path)
mc_loaded.digraph()

In [ ]:
M = DTMC.from_prism_model("datasets/brp.pm",
                          prism_constants={("N",2),("MAX",1)},
                          extra_labels={("uncertain","s=5 & srep=2"),("all","true")})
M.digraph()